In [1]:
import numpy as np
import pandas as pd
train_dat = pd.read_csv('./data/train_stories.csv')
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from random import randint
import nltk.data
from itertools import chain
from nltk.stem.wordnet import WordNetLemmatizer

In [182]:
text = "John was happy. Charlie was angry. She successfully made her gandmother's pizza. Tina went to sleep. George had a dog."
output = ""

In [183]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenized = tokenizer.tokenize(text)
words = word_tokenize(text)
tagged = nltk.pos_tag(words)

In [184]:
tagged

[('John', 'NNP'),
 ('was', 'VBD'),
 ('happy', 'JJ'),
 ('.', '.'),
 ('Charlie', 'NNP'),
 ('was', 'VBD'),
 ('angry', 'JJ'),
 ('.', '.'),
 ('She', 'PRP'),
 ('successfully', 'RB'),
 ('made', 'VBD'),
 ('her', 'PRP$'),
 ('gandmother', 'NN'),
 ("'s", 'POS'),
 ('pizza', 'NN'),
 ('.', '.'),
 ('Tina', 'NNP'),
 ('went', 'VBD'),
 ('to', 'TO'),
 ('sleep', 'VB'),
 ('.', '.'),
 ('George', 'NNP'),
 ('had', 'VBD'),
 ('a', 'DT'),
 ('dog', 'NN'),
 ('.', '.')]

In [185]:
def get_antonyms(input_lemma):
    antonyms = []
    for syn in wn.synsets(input_lemma):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                antonyms.append(lemma.antonyms()[0].name())
    return antonyms

In [187]:
synonyms = wn.synsets('good')
set(chain.from_iterable([word.lemma_names() for word in synonyms]))

{'adept',
 'beneficial',
 'commodity',
 'dear',
 'dependable',
 'effective',
 'estimable',
 'expert',
 'full',
 'good',
 'goodness',
 'honest',
 'honorable',
 'in_effect',
 'in_force',
 'just',
 'near',
 'practiced',
 'proficient',
 'respectable',
 'right',
 'ripe',
 'safe',
 'salutary',
 'secure',
 'serious',
 'skilful',
 'skillful',
 'sound',
 'soundly',
 'thoroughly',
 'trade_good',
 'undecomposed',
 'unspoiled',
 'unspoilt',
 'upright',
 'well'}

In [186]:
get_antonyms('good')

['evil', 'evilness', 'bad', 'badness', 'bad', 'evil', 'ill']

In [188]:
get_antonyms('small')

['large', 'big', 'big']

In [189]:
get_antonyms(words[2])[0]

'unhappy'

In [190]:
output = ""
for i in range(0,len(words)):
    new_word = []
    
    if tagged[i][1] == 'JJ' or tagged[i][1] == 'RB':
        antonyms = get_antonyms(words[i])
        if len(antonyms) > 0:
            new_word = antonyms[0]
        else:
            new_word = words[i]
        
        output = output + " " + new_word        
    
    else:        
        output = output + " " + words[i]
print(output)

 John was unhappy . Charlie was unangry . She unsuccessfully made her gandmother 's pizza . Tina went to sleep . George had a dog .


In [200]:
output = ""
for i in range(0,len(words)):
    new_verb = []
    
    if words[i] == 'was' or words[i] == 'were':
        new_verb = words[i] + " " + "not"
        output = output + " " + new_verb 

        
    elif tagged[i][1] == 'VBD' and words[i] != 'was' and words[i] != 'were':
        new_verb = "did not" + " " + WordNetLemmatizer().lemmatize(words[i],'v')
        output = output + " " + new_verb 
    

    else:
        output = output + " " + words[i]
print(output)

 John was not happy . Charlie was not angry . She successfully did not make her gandmother 's pizza . Tina did not go to sleep . George did not have a dog .


In [2]:
def changeAdj(i, words, tagged, isNegated):
    
    if tagged[i][1] == 'JJ' or tagged[i][1] == 'RB':
        antonyms = get_antonyms(words[i])
        if len(antonyms) > 0:
            new_word = antonyms[0] 
            isNegated = True
        else:
            new_word = words[i]
        
    else:
        new_word = words[i]
    
    return new_word, isNegated

In [3]:
def changeVerb(i, words, tagged, isNegated):
    
    if words[i] == 'was' or words[i] == 'were':
        new_verb = words[i] + " " + "not"
        isNegated= True
        
        
    elif tagged[i][1] == 'VBD' and words[i] != 'was' and words[i] != 'were':
        new_verb = "did not" + " " + WordNetLemmatizer().lemmatize(words[i],'v')
        isNegated = True
    else:
        new_verb = words[i]
    
    return new_verb, isNegated

In [253]:
train_dat = pd.read_csv('./data/train_stories.csv')

In [254]:
train_dat['wsentence5'] = train_dat['sentence5'] 

In [6]:
def negateSentence(sentence):
    words = word_tokenize(sentence)
    tagged = nltk.pos_tag(words)
    isNegated = False
    wordTypes = [word[1] for word in tagged]
    hasAdj = 'JJ' in wordTypes or 'RB' in wordTypes
    
    output = ""
    for i in range(0,len(words)):
        if hasAdj and not isNegated:
            new_word, isNegated = changeAdj(i, words, tagged, isNegated)
        elif not hasAdj and not isNegated:
            new_word, isNegated = changeVerb(i, words, tagged, isNegated)
        else:
            new_word = words[i]
        output = output + " " + new_word  
    return output            
    
        
        
            

In [7]:
for i in range(len(train_dat['sentence5'])):
    train_dat['wsentence5'][i] = negateSentence(train_dat['sentence5'][i])

NameError: name 'get_antonyms' is not defined

In [250]:
negateSentence('The conference became successful when we developed an agenda')

' The conference became unsuccessful when we developed an agenda'

In [258]:
train_dat['wsentence5']

0         Kelly unsuccessfully made a pizza from her gr...
1         His dog leaped and caught it , and he slowly ...
2         He quit his day job and spent all his time gr...
3         Tom and Beth did not grow closer during their...
4                She did not get kicked out in two weeks .
5         Their relationship got good and they no longe...
6               Tom was not paralyzed from the neck down .
7           She did not eat them for every meal that day .
8         Jared discovered his dad 's companionship was...
9                                 She was so embarrassed .
10        The conference became unsuccessful when we de...
11                        Elaine quit the band in a huff .
12        Cathy could not wait to see the skaters at th...
13        The noncommercial convinced Amy and she left ...
14        His dad slowly discovered the monitor brightn...
15                      She is so ungrateful for her job .
16        I breezed through it easily , securing the job

In [259]:
train_dat

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,wsentence5
0,8eb647c6-cdfa-416a-b4a2-09b24f44dcc5,Kelly Made Pizza,Kelly found her grandmother's pizza recipe in ...,Kelly reminisced about how much she loved her ...,Kelly decided that she was going to try to mak...,Kelly studied the recipe and gathered everythi...,Kelly successfully made a pizza from her grand...,Kelly unsuccessfully made a pizza from her gr...
1,e0cab669-993a-41be-978f-97795be07197,Embarrassing Hair,Leo wore a toupee and was anxious about it.,He decided to go out for a short walk.,"It was a very windy day, but he wasn't too con...","Suddenly, a strong wind came through and took ...","His dog leaped and caught it, and he quickly r...","His dog leaped and caught it , and he slowly ..."
2,ad2236b7-de48-45a2-838b-25f166bdee90,Grill Master,Jimmy was a master with his grill.,He spent every weekend getting better with his...,One day he was offered a TV show about grillin...,Jimmy accepted the job in an instant.,He quit his day job and spent all his time gri...,He quit his day job and spent all his time gr...
3,500fe4a4-f7c4-4f46-96ee-e010467b069f,Ice Skating,Tom and Beth were dating.,Tom wanted to go on a fun date.,Tom suggested ice skating.,His date was excited and they went ice skating.,Tom and Beth grew closer during their ice skat...,Tom and Beth did not grow closer during their...
4,522f2bc3-3bb1-4f68-8e99-5b2abd016854,Bad friend 1,I had a friend that I didn't know well but let...,She paid rent then asked for some of it back.,She drinks my juice and eats my food.,She also makes a huge mess and is very sloppy.,She got kicked out in two weeks.,She did not get kicked out in two weeks .
5,03508cbb-1f30-4fce-9788-3523a5a8568b,Sports fanatic,Matthew grew up with a dad that pushed him in ...,He thought he would grow up to be an athlete.,Once he grew up he realized he wanted to do ot...,His dad was very angry.,Their relationship got bad and they no longer ...,Their relationship got good and they no longe...
6,07126538-bbde-413f-b8ca-c8327bf4ea4e,Slick Road,Tom was driving on a rainy night.,The road was slicker than he thought.,Tom hit a watery oily patch.,He spun out of control and hit a tree.,Tom was paralyzed from the neck down.,Tom was not paralyzed from the neck down .
7,63a1ed11-4eb9-4ae8-ad7b-8928f30a36c9,Mango,She had a sudden craving for mangoes.,She went to the store looking for some.,She found some that were very ripe.,The sweet flavor was pungent and satisfying.,She ate them for every meal that day.,She did not eat them for every meal that day .
8,c36e3440-6c49-4338-8283-74ab8da127fc,Jared,Jared liked to fish with his dad every Saturday.,He and his dad would go out early so they coul...,Jared's dad got sick and could no longer go fi...,Jared still got up early on Saturday to hang o...,Jared discovered his dad's companionship was m...,Jared discovered his dad 's companionship was...
9,4835f5c1-cfd7-4559-aa37-36793502f828,Walking in the show,Susie needed models for her catwalk.,So she commissioned Rachel.,Rachel was thrilled.,But the day of the show she tripped.,She was so embarrassed.,She was so embarrassed .


In [260]:
choice = pd.DataFrame(columns=['Option1','Option2'])

choice["Option1"] = train_dat["sentence5"]
choice["Option2"] = train_dat["wsentence5"]

choice = choice.values
#choice.shape (88161, 2)
_ = [np.random.shuffle(i) for i in choice]

choice = pd.DataFrame(choice,
                      columns=['Option1','Option2'])

train_dat["Option1"] = choice["Option1"]
train_dat["Option2"] = choice["Option2"]

train_dat["Correct"] = np.where(train_dat['Option1']==train_dat['sentence5'], '1', '2')

In [261]:
train_dat

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5,wsentence5,Option1,Option2,Correct
0,8eb647c6-cdfa-416a-b4a2-09b24f44dcc5,Kelly Made Pizza,Kelly found her grandmother's pizza recipe in ...,Kelly reminisced about how much she loved her ...,Kelly decided that she was going to try to mak...,Kelly studied the recipe and gathered everythi...,Kelly successfully made a pizza from her grand...,Kelly unsuccessfully made a pizza from her gr...,Kelly unsuccessfully made a pizza from her gr...,Kelly successfully made a pizza from her grand...,2
1,e0cab669-993a-41be-978f-97795be07197,Embarrassing Hair,Leo wore a toupee and was anxious about it.,He decided to go out for a short walk.,"It was a very windy day, but he wasn't too con...","Suddenly, a strong wind came through and took ...","His dog leaped and caught it, and he quickly r...","His dog leaped and caught it , and he slowly ...","His dog leaped and caught it , and he slowly ...","His dog leaped and caught it, and he quickly r...",2
2,ad2236b7-de48-45a2-838b-25f166bdee90,Grill Master,Jimmy was a master with his grill.,He spent every weekend getting better with his...,One day he was offered a TV show about grillin...,Jimmy accepted the job in an instant.,He quit his day job and spent all his time gri...,He quit his day job and spent all his time gr...,He quit his day job and spent all his time gri...,He quit his day job and spent all his time gr...,1
3,500fe4a4-f7c4-4f46-96ee-e010467b069f,Ice Skating,Tom and Beth were dating.,Tom wanted to go on a fun date.,Tom suggested ice skating.,His date was excited and they went ice skating.,Tom and Beth grew closer during their ice skat...,Tom and Beth did not grow closer during their...,Tom and Beth grew closer during their ice skat...,Tom and Beth did not grow closer during their...,1
4,522f2bc3-3bb1-4f68-8e99-5b2abd016854,Bad friend 1,I had a friend that I didn't know well but let...,She paid rent then asked for some of it back.,She drinks my juice and eats my food.,She also makes a huge mess and is very sloppy.,She got kicked out in two weeks.,She did not get kicked out in two weeks .,She got kicked out in two weeks.,She did not get kicked out in two weeks .,1
5,03508cbb-1f30-4fce-9788-3523a5a8568b,Sports fanatic,Matthew grew up with a dad that pushed him in ...,He thought he would grow up to be an athlete.,Once he grew up he realized he wanted to do ot...,His dad was very angry.,Their relationship got bad and they no longer ...,Their relationship got good and they no longe...,Their relationship got bad and they no longer ...,Their relationship got good and they no longe...,1
6,07126538-bbde-413f-b8ca-c8327bf4ea4e,Slick Road,Tom was driving on a rainy night.,The road was slicker than he thought.,Tom hit a watery oily patch.,He spun out of control and hit a tree.,Tom was paralyzed from the neck down.,Tom was not paralyzed from the neck down .,Tom was not paralyzed from the neck down .,Tom was paralyzed from the neck down.,2
7,63a1ed11-4eb9-4ae8-ad7b-8928f30a36c9,Mango,She had a sudden craving for mangoes.,She went to the store looking for some.,She found some that were very ripe.,The sweet flavor was pungent and satisfying.,She ate them for every meal that day.,She did not eat them for every meal that day .,She ate them for every meal that day.,She did not eat them for every meal that day .,1
8,c36e3440-6c49-4338-8283-74ab8da127fc,Jared,Jared liked to fish with his dad every Saturday.,He and his dad would go out early so they coul...,Jared's dad got sick and could no longer go fi...,Jared still got up early on Saturday to hang o...,Jared discovered his dad's companionship was m...,Jared discovered his dad 's companionship was...,Jared discovered his dad's companionship was m...,Jared discovered his dad 's companionship was...,1
9,4835f5c1-cfd7-4559-aa37-36793502f828,Walking in the show,Susie needed models for her catwalk.,So she commissioned Rachel.,Rachel was thrilled.,But the day of the show she tripped.,She was so em

In [262]:
train_dat.to_csv("./data/train_dat_wrongchoice.csv", sep=',', encoding='utf-8',index = False)